In [171]:
EPS = 1e-10

def eq_zero(x):
    return abs(x) < EPS

def lt_zero(x):
    return x < -EPS

def gt_zero(x):
    return x > EPS

In [172]:
def bisection(f, l, r, iterations=100):
    assert l <= r
    if eq_zero(f(l)):
        return l
    if eq_zero(f(r)):
        return r
    assert lt_zero(f(l) * f(r))
    for _ in range(iterations):
        m = (l + r) / 2
        if eq_zero(f(m)):
            return m
        if lt_zero(f(l) * f(m)):
            r = m
        else:
            l = m
    return l

def newton(f, df, x0=0, iterations=100):
    x = x0
    for _ in range(iterations):
        x -= f(x) / df(x)
    return x

def root(a, k, method):
    assert method in ['bisection', 'newton']
    assert k > 0 and (gt_zero(a) or k % 2 == 1)
    if method == 'bisection':
        l, r = 0., max(1., abs(a))
        if lt_zero(a):
            l, r = -r, -l
        return bisection(lambda x: x ** k - a, l, r)
    if method == 'newton':
        f = lambda x: x ** k - a
        df = lambda x: k * x ** (k - 1)
        return newton(f, df, a)

def show_root(a, k, method):
    print(f'{method}: {k}th root of {a} is {root(a, k, method)}')

show_root(a=-.001, k=3, method='bisection')
show_root(a=100, k=2, method='bisection')
show_root(a=-239.30, k=1, method='bisection')
show_root(a=-.001, k=3, method='newton')
show_root(a=100, k=2, method='newton')
show_root(a=-239.30, k=1, method='newton')

bisection: 3th root of -0.001 is -0.099609375
bisection: 2th root of 100 is 10.000000894069672
bisection: 1th root of -239.3 is -239.3
newton: 3th root of -0.001 is -0.1
newton: 2th root of 100 is 10.0
newton: 1th root of -239.3 is -239.3


In [173]:
C = 100000

def get_intervals_with_roots(f, l, r):
    assert l < r
    intervals = []
    ls = np.linspace(l, r, C)
    for i in range(C - 1):
        ll, rr = ls[i], ls[i + 1]
        if lt_zero(f(rr) * f(ll)):
            intervals.append((ll, rr))
    return intervals

p = lambda coefs, x: sum(c * x ** i for i, c in enumerate(coefs[::-1]))
dp = lambda coefs, x: sum((i + 1) * c * x ** i for i, c in enumerate(coefs[::-1][:-1]))

L, R = -3, 3
coefs = [1, 1, -10, -10, 21, 21]
p_coefs = lambda x: p(coefs, x)
intervals = get_intervals_with_roots(p_coefs, L, R)
intervals

[(-2.6457564575645756, -2.64569645696457),
 (-1.7320673206732067, -1.7320073200732007),
 (1.7320073200732002, 1.732067320673207),
 (2.6456964569645693, 2.645756457564575)]

In [174]:
print('roots:')
for i in intervals:
    print(bisection(p_coefs, *i))

roots:
-2.645750832508325
-1.7320485704857047
1.7320513830138302
2.6457517700177


In [175]:
dp_coefs = lambda x: dp(coefs, x)
candidates = [L, R]
candidates += [bisection(dp_coefs, *i) for i in  get_intervals_with_roots(dp_coefs, L, R)]
argmin = min(candidates, key=p_coefs)
print(f'global minima is {p_coefs(argmin)} at {argmin}')

global minima is -24 at -3


In [176]:
f = lambda a, b, c, d, x: np.exp(a * x) + np.exp(-b * x) + c * (x - d) ** 2
df = lambda a, b, c, d, x: a * np.exp(a * x) - b * np.exp(-b * x) + 2 * c * (x - d)
d2f = lambda a, b, c, d, x: a ** 2 * np.exp(a * x) + b ** 2 * np.exp(-b * x) + 2 * c

a, b, c, d = 1, 2, 3, -7
f_abcd = lambda x: f(a, b, c, d, x)
df_abcd = lambda x: df(a, b, c, d, x)
d2f_abcd = lambda x: d2f(a, b, c, d, x)

L, R = -100, 100
print('bisection:', f_abcd(bisection(df_abcd, L, R)))
print('newton:', f_abcd(newton(df_abcd, d2f_abcd)))

l, r = L, R
while gt_zero(r - l):
    ll, rr = (2 * l + r) / 3, (2 * r + l) / 3
    if f_abcd(ll) > f_abcd(rr):
        l = ll
    else:
        r = rr
print('ternar:', f_abcd(l))

bisection: 110.76493266641712
newton: 110.76493266641181
ternar: 110.76493266641184
